## Description
I was working with one third part of dataset (851968 signals), I found myself running into problems with memory shortages on Kaggle’s default and decided to use powerful Google Cloud Platform virtual machine instance.
I used the virtual machine instance n1-standard-8 (8 vCPU, 30 GB memory) with a NVIDIA Tesla V100 GPU to run my training session.
This required about 7.5 hours of training time

In [1]:
# !aws s3 sync s3://mlflow_sber/Data data --endpoint-url http://hb.bizmrg.com

## Import

In [2]:
import os
import sys
import json
import keras
import warnings
import requests
import numpy as np 
import pandas as pd 
import itertools
from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.models import Model, load_model
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

2022-03-21 14:17:39.212887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-03-21 14:17:39.212974: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Constants

In [3]:
for dirname, _, filenames in os.walk('data'):    
    if not 'data/weights' in dirname:
        files = {}
        for filename in filenames:        
            files[filename] = os.path.join(dirname, filename)
# print(files)

# Open classes.txt
with open(files['classes.txt']) as file: 

   classes = file.read()

def str_to_list(line):
    line = line.replace('\n','')
    line = line.replace('classes = ','')
    line = eval(line)
    return line
    
classes = str_to_list(classes)
print(classes)

with open(files['input_example.txt']) as file: 
    input_example = file.read()
    input_example = input_example.replace('\n','')

['32PSK', '16APSK', '32QAM', 'FM', 'GMSK', '32APSK', 'OQPSK', '8ASK', 'BPSK', '8PSK', 'AM-SSB-SC', '4ASK', '16PSK', '64APSK', '128QAM', '128APSK', 'AM-DSB-SC', 'AM-SSB-WC', '64QAM', 'QPSK', '256QAM', 'AM-DSB-WC', 'OOK', '16QAM']


## Load data

In [4]:
general_path = 'data'
# Upload signals, labels, snrs
labels = np.load(f'{general_path}/labels.npy', mmap_mode = 'r')
signals = np.load(f'{general_path}/signals.npy', mmap_mode = 'r')
snrs = np.load(f'{general_path}/snrs.npy', mmap_mode = 'r')

train_acc = np.load(f'{general_path}/train_acc.npy', mmap_mode = 'r')
val_acc = np.load(f'{general_path}/val_acc.npy', mmap_mode = 'r')
train_loss = np.load(f'{general_path}/train_loss.npy', mmap_mode = 'r')
val_loss = np.load(f'{general_path}/val_loss.npy', mmap_mode = 'r')

# Upload model
model_full = load_model(files['model_full_SNR.h5'])

2022-03-21 14:17:47.048489: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-21 14:17:47.048612: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mlflow-ml): /proc/driver/nvidia/version does not exist
2022-03-21 14:17:47.050187: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Preprocessing

In [5]:
# Split arrays for 3 parts (we take only third part of dataset of labeled signals because of the memory)
part = 8
signals = signals[::part, :, :]
labels = labels[::part, :]
snrs = snrs[::part, :]

print(signals.shape)
print(labels.shape)
print(snrs.shape)

# Ndarray to array
snrs = np.ravel(snrs)
print(f"All possible SNRS: {np.unique(snrs)} db")

# Masked numpy array 
c = np.ma.masked_where(snrs > 8, snrs)
msk = c.mask
# Count unique elements in array
print(f'Unique elements in array {np.unique(c.mask, return_counts=True)}')

# Mask array of signals and labels (snrs > 8)
signals = signals[msk]
labels = labels[msk]

print(f'Quantity of signals {len(signals)}')

(319488, 1024, 2)
(319488, 24)
(319488, 1)
All possible SNRS: [-20. -18. -16. -14. -12. -10.  -8.  -6.  -4.  -2.   0.   2.   4.   6.
   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.  28.  30.] db
Unique elements in array (array([False,  True]), array([184320, 135168]))
Quantity of signals 135168


In [ ]:
# Train|test = 80|20

x_train,x_test, y_train, y_test = train_test_split(signals, labels, train_size=0.8, stratify=labels, random_state = 42)


print(x_test.shape)
print(y_test.shape)

# Train|validation|test = 64|16|20
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, stratify=y_train, random_state = 42)


print(x_train.shape)
print(y_train.shape)

## Evaluate Model on Test Data

In [ ]:
loss, acc = model_full.evaluate(x_test, y_test, batch_size=32)
print('EVALUATING MODEL ON TEST DATA:')
print('Test Accuracy: ', str(round(acc*100, 2)), '%')
print('\n')

The model achieved maximum accuracy of 95.72% on the clean signal dataset (SNR > 8dB).

*Note that the maximum classification accuracy of 62% if you want to test the model to its absolute limits on a mix of clean signals and signals with very high interference. Some of the signals have so much noise that they are virtually unrecognizable.*

## Confusion Matrix

In [ ]:
predictions=model_full.predict(x_test)
classes_y=np.argmax(predictions,axis=1)
conf_matrix = confusion_matrix(y_true=np.argmax(y_test,axis=1), y_pred=classes_y)

In [ ]:
nr_rows = conf_matrix.shape[0]
nr_cols = conf_matrix.shape[1]

plt.figure(figsize=(18,18), dpi= 200) 
im = plt.imshow(conf_matrix, cmap=plt.cm.Greens)
ax = plt.gca()
plt.title('Confusion Matrix', fontsize=16) 
plt.ylabel('Actual Labels', fontsize=12)
plt.xlabel('Predicted Labels', fontsize=12)
tick_marks = np.arange(len(classes))
plt.yticks(tick_marks, classes)
# plt.xticks(tick_marks, classes)

for i, j in itertools.product(range(nr_rows), range(nr_cols)):
    plt.text(j, i, conf_matrix[i, j], horizontalalignment='center',
            color='white' if conf_matrix[i, j] > conf_matrix.max()/2 else 'black')
    

# Divide existing axes and create new axes
# at bottom side of image
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax) 

## Conclusions
Reaching maximum accuracy of 62% on a mix of clean signals and signals with very high interference and 95.72% on the clean signal dataset, the model isn't perfect.

In [ ]:
history = [[train_acc, val_acc],[train_loss, val_loss]]
title = ['Model accuracy', 'Model loss']
ylabel = ['accuracy', 'loss']
fig = plt.figure(figsize=(20, 7), dpi=80)
for i, id in enumerate(title):
    plt.subplot(1, 2, i+1)
   
    plt.plot(history[i][0], label = 'train')
    plt.plot(history[i][1], label = 'val')
    plt.title(id)
    plt.xlabel('epoch')
    plt.ylabel(ylabel[i])
    plt.legend(loc='upper right')
plt.tight_layout(pad=1.7)
    

Below is an example of a signal that was mis-classified by the model (AM-SSB-WC signal was mis-classified as a 64QAM signal)

In [ ]:
idx = [17,18]
fig = plt.figure(figsize=(20, 5), dpi=80)
for i, id in enumerate(idx):
    plt.subplot(1, 2, i+1)
    plt.plot(signals[id][:, 0], color='green', label='I component')
    plt.plot(signals[id][:, 1], color='salmon', label='Q component')
    plt.title(classes[id])
    plt.xlabel('Points')
    plt.ylabel('Amplitude')
    plt.legend(loc='upper right')
plt.tight_layout(pad=1.7)

## Simple predict (*full model)

In [ ]:
class_names = classes.copy() 

# ['32PSK', '16APSK', '32QAM', 'FM', 'GMSK', '32APSK', 'OQPSK', '8ASK',
#           'BPSK', '8PSK', 'AM-SSB-SC', '4ASK', '16PSK', '64APSK', '128QAM', '128APSK', 
#           'AM-DSB-SC', 'AM-SSB-WC', '64QAM', 'QPSK', '256QAM', 'AM-DSB-WC', 'OOK', '16QAM']

sig = 15790
predict = model_full.predict(signals[sig][None, :])
print(f'Predicted class is {classes[np.argmax(predict[0])]}')

## Test serving model

In [ ]:
answer = !curl -X POST -H "Content-Type:application/json" --data '{"inputs": [[[-0.9107131361961365, -0.3592507243156433], [-0.6384691596031189, -0.2383548766374588], [-0.3330206871032715, -0.13245554268360138], [-0.00996798649430275, -0.0012351353652775288], [0.30146822333335876, 0.109389528632164], [0.5885838270187378, 0.24158519506454468], [0.8464970588684082, 0.33450713753700256], [1.0707149505615234, 0.4238452911376953], [1.2340348958969116, 0.4950251877307892], [1.363065242767334, 0.5458983182907104], [1.4335614442825317, 0.5720593929290771], [1.4261561632156372, 0.5657274723052979], [1.382374882698059, 0.5538820624351501], [1.2786179780960083, 0.5213793516159058], [1.1299725770950317, 0.45998579263687134], [0.9583629965782166, 0.3868354856967926], [0.7561097741127014, 0.3016241788864136], [0.5199660658836365, 0.21662552654743195], [0.27331241965293884, 0.09417765587568283], [0.01572216860949993, 0.001327180303633213], [-0.2481185495853424, -0.11116272211074829], [-0.5217671990394592, -0.20214387774467468], [-0.7475267648696899, -0.3041779696941376], [-0.9684640169143677, -0.3918049931526184], [-1.144866943359375, -0.4554829001426697], [-1.286524772644043, -0.5252783894538879], [-1.3905473947525024, -0.549318253993988], [-1.4112226963043213, -0.577201247215271], [-1.4064332246780396, -0.5707699656486511], [-1.3213818073272705, -0.5392974615097046], [-1.2049669027328491, -0.48922258615493774], [-1.0207464694976807, -0.4225119948387146], [-0.801733672618866, -0.32653889060020447], [-0.5624220967292786, -0.23736824095249176], [-0.2980879545211792, -0.1259581446647644], [-0.00869764108210802, 0.0034572097938507795], [0.26933637261390686, 0.11165649443864822], [0.5360670685768127, 0.21675291657447815], [0.8028290867805481, 0.3249829411506653], [1.0135024785995483, 0.4115450978279114], [1.1928119659423828, 0.4859805703163147], [1.332851767539978, 0.5449128150939941], [1.4063386917114258, 0.5735136866569519], [1.434728980064392, 0.5910016894340515], [1.381662130355835, 0.5763167142868042], [1.3090293407440186, 0.5421664714813232], [1.1940628290176392, 0.4955401122570038], [1.0153608322143555, 0.41813963651657104], [0.7881900072097778, 0.32954642176628113], [0.5531723499298096, 0.2313782125711441], [0.2994937598705292, 0.11696834862232208], [0.007216433994472027, 0.0037063960917294025], [-0.2614273428916931, -0.11599403619766235], [-0.5201118588447571, -0.21807637810707092], [-0.7813454270362854, -0.32157209515571594], [-0.9898155927658081, -0.409047394990921], [-1.1690810918807983, -0.4830685257911682], [-1.297783613204956, -0.5512694716453552], [-1.3806188106536865, -0.5803098678588867], [-1.4235568046569824, -0.589305579662323], [-1.3990519046783447, -0.5778487920761108], [-1.3147714138031006, -0.5520553588867188], [-1.1788883209228516, -0.4930739104747772], [-1.0089432001113892, -0.4213714897632599], [-0.8008131980895996, -0.3341924548149109], [-0.5537225008010864, -0.23792403936386108], [-0.28941476345062256, -0.11606042832136154], [-0.0031424066983163357, -0.008774051442742348], [0.2529889941215515, 0.10530731827020645], [0.539016842842102, 0.23585626482963562], [0.7797582149505615, 0.3249678611755371], [0.9807771444320679, 0.4328780174255371], [1.1676982641220093, 0.5002671480178833], [1.3054031133651733, 0.5559789538383484], [1.388542890548706, 0.5910506844520569], [1.408001184463501, 0.6009323000907898], [1.3774882555007935, 0.5912101864814758], [1.307089924812317, 0.5492448806762695], [1.1866954565048218, 0.49891096353530884], [1.0128334760665894, 0.4358319640159607], [0.7904513478279114, 0.32864487171173096], [0.541933536529541, 0.21573835611343384], [0.28409162163734436, 0.11797143518924713], [0.02235225960612297, -0.005696091800928116], [-0.2588609755039215, -0.11404368281364441], [-0.5361744165420532, -0.23647736012935638], [-0.7804598808288574, -0.3355799913406372], [-1.004587173461914, -0.4173394441604614], [-1.1708343029022217, -0.5044108033180237], [-1.3104702234268188, -0.5582373142242432], [-1.3919755220413208, -0.5992332100868225], [-1.4062286615371704, -0.6113119125366211], [-1.400315761566162, -0.5944411158561707], [-1.2969465255737305, -0.5663411617279053], [-1.18818998336792, -0.5001252293586731], [-1.008832335472107, -0.43862172961235046], [-0.7894243001937866, -0.34802934527397156], [-0.5435665249824524, -0.24652913212776184], [-0.2872009873390198, -0.12589921057224274], [-0.006486743222922087, 0.0010495905298739672], [0.26607581973075867, 0.11589109897613525], [0.5307813286781311, 0.23055501282215118], [0.7783848643302917, 0.34332340955734253], [1.0007084608078003, 0.4384863078594208], [1.1673389673233032, 0.5077624917030334], [1.2857905626296997, 0.5680974125862122], [1.382181167602539, 0.6031900644302368], [1.413243055343628, 0.6269596219062805], [1.3810031414031982, 0.5971390008926392], [1.3049888610839844, 0.5677787661552429], [1.1857905387878418, 0.5114778876304626], [1.007043480873108, 0.4442921578884125], [0.7835078239440918, 0.34843015670776367], [0.5369248986244202, 0.2368747889995575], [0.2767019271850586, 0.122355155646801], [0.005350351333618164, 0.004801636561751366], [-0.26942679286003113, -0.13090699911117554], [-0.5254408717155457, -0.2367973029613495], [-0.7659979462623596, -0.3448503911495209], [-0.9869321584701538, -0.44463488459587097], [-1.1617132425308228, -0.5107852220535278], [-1.2900564670562744, -0.5812703371047974], [-1.3803809881210327, -0.6134179830551147], [-1.4003973007202148, -0.6170243620872498], [-1.3670775890350342, -0.6129717230796814], [-1.3098803758621216, -0.570750892162323], [-1.175673246383667, -0.5248543620109558], [-0.9961920380592346, -0.4613879323005676], [-0.7854982614517212, -0.3394617736339569], [-0.5619885325431824, -0.24087591469287872], [-0.2791515290737152, -0.12121769040822983], [-0.011426196433603764, -0.009762544184923172], [0.26383712887763977, 0.11013413220643997], [0.5353081822395325, 0.23569045960903168], [0.7627503275871277, 0.34485289454460144], [0.9732697606086731, 0.4372217357158661], [1.1502653360366821, 0.5199276208877563], [1.2850676774978638, 0.5734134912490845], [1.3730992078781128, 0.6147181987762451], [1.3944612741470337, 0.6339530944824219], [1.3688602447509766, 0.6143905520439148], [1.2995424270629883, 0.5872892737388611], [1.1793595552444458, 0.5353761911392212], [0.9972180128097534, 0.43857696652412415], [0.7839193344116211, 0.3545304536819458], [0.5512893199920654, 0.2391010969877243], [0.28034988045692444, 0.1311810463666916], [0.005674105603247881, 0.007782700937241316], [-0.26125118136405945, -0.11948645114898682], [-0.5149866342544556, -0.22767120599746704], [-0.7695817351341248, -0.34335872530937195], [-0.9872852563858032, -0.43471264839172363], [-1.1501314640045166, -0.5268574357032776], [-1.2883508205413818, -0.5994025468826294], [-1.3709088563919067, -0.6182401776313782], [-1.4002748727798462, -0.6358012557029724], [-1.3737488985061646, -0.6286522746086121], [-1.2904313802719116, -0.5904059410095215], [-1.1630202531814575, -0.5416451692581177], [-0.9781783223152161, -0.4630280137062073], [-0.7794892191886902, -0.36570075154304504], [-0.54463791847229, -0.2508159577846527], [-0.2771569788455963, -0.14333966374397278], [-0.012366807088255882, -0.008496641181409359], [0.25114887952804565, 0.11325360089540482], [0.5164262056350708, 0.2420376092195511], [0.7661231160163879, 0.3551732003688812], [0.9690171480178833, 0.44251757860183716], [1.1518492698669434, 0.5268933176994324], [1.2805322408676147, 0.5913645029067993], [1.357430338859558, 0.6231346130371094], [1.390870451927185, 0.6435052752494812], [1.358506441116333, 0.6262770295143127], [1.2841737270355225, 0.5995863676071167], [1.14643132686615, 0.5333662629127502], [0.9904919266700745, 0.45563966035842896], [0.7887681126594543, 0.35197213292121887], [0.5297102928161621, 0.2517784535884857], [0.27230942249298096, 0.1273067593574524], [0.01494411937892437, -0.000416566472267732], [-0.2688900828361511, -0.11370345205068588], [-0.5184051394462585, -0.2368309050798416], [-0.7556903958320618, -0.3475874960422516], [-0.9553634524345398, -0.4520190954208374], [-1.1517325639724731, -0.5360611081123352], [-1.2725062370300293, -0.59980708360672], [-1.3524500131607056, -0.6276601552963257], [-1.3762974739074707, -0.646506667137146], [-1.348628282546997, -0.6337531208992004], [-1.2776110172271729, -0.6058498024940491], [-1.1317895650863647, -0.5421107411384583], [-0.9830712676048279, -0.4715060591697693], [-0.7746374011039734, -0.3655136525630951], [-0.5283949971199036, -0.2558661699295044], [-0.2832988202571869, -0.14559413492679596], [-0.009491510689258575, -6.757237861165777e-05], [0.2564537525177002, 0.12094025313854218], [0.5222353935241699, 0.23603999614715576], [0.7595910429954529, 0.36455464363098145], [0.962653636932373, 0.44564884901046753], [1.13685941696167, 0.5402026176452637], [1.273735523223877, 0.5973576903343201], [1.3372387886047363, 0.6440989375114441], [1.3640215396881104, 0.6539246439933777], [1.353475570678711, 0.6475399732589722], [1.2672092914581299, 0.6003252267837524], [1.149469017982483, 0.5467402338981628], [0.9786678552627563, 0.46368086338043213], [0.7706162333488464, 0.36257606744766235], [0.526848554611206, 0.25621041655540466], [0.27397966384887695, 0.12992557883262634], [0.009315735660493374, -0.006758647505193949], [-0.2598816752433777, -0.11877107620239258], [-0.5174725651741028, -0.24739839136600494], [-0.7547680139541626, -0.35335084795951843], [-0.955114483833313, -0.4577457308769226], [-1.1375123262405396, -0.5448554158210754], [-1.2567756175994873, -0.6104846000671387], [-1.3383978605270386, -0.6410585045814514], [-1.3588873147964478, -0.6657418608665466], [-1.3340145349502563, -0.6441290378570557], [-1.266419768333435, -0.6142867803573608], [-1.1337848901748657, -0.5513101816177368], [-0.9817368388175964, -0.4712689220905304], [-0.7633987069129944, -0.36707139015197754], [-0.5277393460273743, -0.2587651014328003], [-0.27191781997680664, -0.14345411956310272], [-0.01142861694097519, -0.00869603082537651], [0.2645457983016968, 0.12218925356864929], [0.5140357613563538, 0.25943031907081604], [0.747383713722229, 0.3612035810947418], [0.9537287950515747, 0.46728137135505676], [1.1213102340698242, 0.5487260818481445], [1.250049352645874, 0.6132208108901978], [1.3286080360412598, 0.6575299501419067], [1.3497081995010376, 0.6590248942375183], [1.323251724243164, 0.6621424555778503], [1.2574468851089478, 0.6033706665039062], [1.1307083368301392, 0.5559822916984558], [0.9619718194007874, 0.46443986892700195], [0.7611029744148254, 0.37410229444503784], [0.5340170860290527, 0.2520477771759033], [0.2715953588485718, 0.13616207242012024], [-0.00022595521295443177, 0.006011182442307472], [-0.2533586621284485, -0.12693029642105103], [-0.5066521167755127, -0.2487393021583557], [-0.7374706268310547, -0.3672976493835449], [-0.9382040500640869, -0.4663102626800537], [-1.1253937482833862, -0.5564144253730774], [-1.2440069913864136, -0.6309285163879395], [-1.331551194190979, -0.6619203686714172], [-1.338904619216919, -0.6656655669212341], [-1.32735276222229, -0.6589164137840271], [-1.2462067604064941, -0.6153154969215393], [-1.1269502639770508, -0.5579891204833984], [-0.9532455801963806, -0.46856990456581116], [-0.7500511407852173, -0.3766692876815796], [-0.5192654132843018, -0.2517872452735901], [-0.2705913484096527, -0.13622520864009857], [-0.015751395374536514, -0.003981384914368391], [0.25436678528785706, 0.11847677081823349], [0.49913838505744934, 0.24703848361968994], [0.7369839549064636, 0.3650789260864258], [0.9232326149940491, 0.4677249789237976], [1.109023094177246, 0.5596910119056702], [1.2258403301239014, 0.6212647557258606], [1.3111597299575806, 0.6469113230705261], [1.3424354791641235, 0.6657683253288269], [1.311123013496399, 0.648952066898346], [1.2308192253112793, 0.6314429640769958], [1.1123751401901245, 0.5601732134819031], [0.9443421363830566, 0.4720468819141388], [0.7500115633010864, 0.38547831773757935], [0.5082477331161499, 0.2636265754699707], [0.26911666989326477, 0.12166101485490799], [0.0037087525706738234, 0.009909039363265038], [-0.25006380677223206, -0.12793661653995514], [-0.4931182563304901, -0.250680536031723], [-0.7353647351264954, -0.37128257751464844], [-0.9199151396751404, -0.4849196672439575], [-1.101342797279358, -0.5354169011116028], [-1.2161760330200195, -0.6252553462982178], [-1.2952998876571655, -0.6724154949188232], [-1.3235448598861694, -0.6756318211555481], [-1.303728699684143, -0.6663011908531189], [-1.2222176790237427, -0.6199709177017212], [-1.1076774597167969, -0.5692222118377686], [-0.9347566366195679, -0.46830877661705017], [-0.7432732582092285, -0.390154629945755], [-0.5112924575805664, -0.26112091541290283], [-0.2707500159740448, -0.14890433847904205], [-0.013718295842409134, -0.0030103193130344152], [0.24590902030467987, 0.12411144375801086], [0.4927869737148285, 0.2577405869960785], [0.7195340991020203, 0.36574119329452515], [0.9275720715522766, 0.4711931645870209], [1.091981291770935, 0.5631089806556702], [1.208681344985962, 0.6376426815986633], [1.292083501815796, 0.6611207723617554], [1.3101139068603516, 0.6794192790985107], [1.2916101217269897, 0.6576292514801025], [1.2095211744308472, 0.6268275380134583], [1.0924855470657349, 0.5648234486579895], [0.9436468482017517, 0.4860915243625641], [0.7386912703514099, 0.3841765820980072], [0.5089952945709229, 0.2601407766342163], [0.27338266372680664, 0.14053656160831451], [0.008757179602980614, 0.00243106996640563], [-0.23818811774253845, -0.12371329218149185], [-0.49423959851264954, -0.2536245286464691], [-0.7278949022293091, -0.3784162998199463], [-0.918624222278595, -0.4680216908454895], [-1.0730371475219727, -0.5677345991134644], [-1.1935195922851562, -0.6223291754722595], [-1.2767090797424316, -0.6638716459274292], [-1.3104804754257202, -0.6819416284561157], [-1.2813355922698975, -0.6708717942237854], [-1.1997827291488647, -0.6334514617919922], [-1.0888371467590332, -0.5755802989006042], [-0.9227780103683472, -0.4853932857513428], [-0.7340273857116699, -0.3864166736602783], [-0.5180058479309082, -0.26646068692207336], [-0.26569506525993347, -0.13595974445343018], [-0.01006849855184555, -0.008252130821347237], [0.2461455762386322, 0.11755833029747009], [0.49127882719039917, 0.2487117052078247], [0.7174313068389893, 0.37229666113853455], [0.9083406925201416, 0.4696720242500305], [1.0754891633987427, 0.5631449222564697], [1.1881898641586304, 0.6306372284889221], [1.2656126022338867, 0.6690678596496582], [1.2791365385055542, 0.6979590654373169], [1.2666327953338623, 0.6628650426864624], [1.1920267343521118, 0.6280065178871155], [1.0780835151672363, 0.5777074694633484], [0.927051305770874, 0.488835871219635], [0.7236341834068298, 0.3835029900074005], [0.49078449606895447, 0.2633109986782074], [0.24909406900405884, 0.13511185348033905], [-0.0007611909531988204, 0.011159065179526806], [-0.23327802121639252, -0.13200171291828156], [-0.4840690493583679, -0.25289297103881836], [-0.7094596028327942, -0.37451109290122986], [-0.901002824306488, -0.472603440284729], [-1.0705183744430542, -0.5727770328521729], [-1.1823029518127441, -0.6346117854118347], [-1.237096905708313, -0.6663750410079956], [-1.2786424160003662, -0.6870852112770081], [-1.2444828748703003, -0.6686901450157166], [-1.1714260578155518, -0.6399837136268616], [-1.0656516551971436, -0.5806255340576172], [-0.9067137837409973, -0.492333322763443], [-0.72757488489151, -0.39077115058898926], [-0.49282196164131165, -0.26572075486183167], [-0.25528016686439514, -0.13849256932735443], [-0.008883800357580185, -0.002896753838285804], [0.2394767552614212, 0.12440023571252823], [0.4732325077056885, 0.2537679672241211], [0.696181058883667, 0.37600746750831604], [0.8947206139564514, 0.4769194722175598], [1.0437783002853394, 0.5564984083175659], [1.1616498231887817, 0.6407585740089417], [1.2298791408538818, 0.674818217754364], [1.2649999856948853, 0.6931549310684204], [1.2368175983428955, 0.6834231615066528], [1.164583444595337, 0.6399387121200562], [1.0456379652023315, 0.585574746131897], [0.8948265314102173, 0.4774390757083893], [0.6945787668228149, 0.38562434911727905], [0.485691100358963, 0.28245043754577637], [0.24752840399742126, 0.13712462782859802], [0.015608218498528004, -0.0009955867426469922], [-0.2312631607055664, -0.1364353448152542], [-0.45589616894721985, -0.2575114965438843], [-0.6874834895133972, -0.3809947967529297], [-0.8729725480079651, -0.48867279291152954], [-1.0368787050247192, -0.5769320726394653], [-1.1543153524398804, -0.636661946773529], [-1.2230647802352905, -0.6697961091995239], [-1.2579489946365356, -0.6916688084602356], [-1.2257800102233887, -0.6781775951385498], [-1.156536340713501, -0.6436724066734314], [-1.047821044921875, -0.5830587148666382], [-0.8879046440124512, -0.48240017890930176], [-0.699884295463562, -0.3947800397872925], [-0.4912797510623932, -0.2600325644016266], [-0.250296026468277, -0.14598321914672852], [0.0073257191106677055, 0.006331728305667639], [0.22875994443893433, 0.11912591755390167], [0.4631400406360626, 0.26001933217048645], [0.6892147064208984, 0.3808497488498688], [0.8761263489723206, 0.47891587018966675], [1.0277214050292969, 0.5748143196105957], [1.1535634994506836, 0.6438436508178711], [1.207682728767395, 0.6797290444374084], [1.2457674741744995, 0.677365243434906], [1.2189875841140747, 0.684482753276825], [1.158675193786621, 0.6427388787269592], [1.0481616258621216, 0.5713486075401306], [0.8867517709732056, 0.4948684275150299], [0.6892197728157043, 0.3904951512813568], [0.4832885265350342, 0.27731144428253174], [0.2513309717178345, 0.13987232744693756], [0.005984220653772354, -0.0037493628915399313], [-0.2323509156703949, -0.1441797912120819], [-0.45915231108665466, -0.2625650465488434], [-0.6853065490722656, -0.3687702715396881], [-0.8656928539276123, -0.4889381229877472], [-1.0218640565872192, -0.5710839629173279], [-1.1432925462722778, -0.6447404026985168], [-1.2137348651885986, -0.6806652545928955], [-1.2325791120529175, -0.6948410272598267], [-1.2092632055282593, -0.6874371767044067], [-1.142513632774353, -0.6457909345626831], [-1.031983733177185, -0.5759792327880859], [-0.8809967637062073, -0.4974117875099182], [-0.6789946556091309, -0.39153677225112915], [-0.4751841425895691, -0.271538108587265], [-0.2539944648742676, -0.14161330461502075], [-0.01578570529818535, 5.600254371529445e-05], [0.23700426518917084, 0.13744910061359406], [0.45763325691223145, 0.2675037086009979], [0.677415668964386, 0.375654935836792], [0.8593539595603943, 0.4858616292476654], [1.0163217782974243, 0.5741782188415527], [1.1265736818313599, 0.6489385366439819], [1.2063506841659546, 0.6891952753067017], [1.230688452720642, 0.7099207639694214], [1.2051647901535034, 0.6843326091766357], [1.133387804031372, 0.6419779658317566], [1.0258697271347046, 0.5828716158866882], [0.8666213154792786, 0.4934512674808502], [0.686575174331665, 0.3906513452529907], [0.47573933005332947, 0.27026844024658203], [0.2351759672164917, 0.13719461858272552], [0.0054665603674948215, 0.008469617925584316], [-0.2253018617630005, -0.13509267568588257], [-0.46122077107429504, -0.26674023270606995], [-0.669739842414856, -0.3918440341949463], [-0.8489349484443665, -0.488627552986145], [-1.009382963180542, -0.582527756690979], [-1.1163196563720703, -0.6471818685531616], [-1.1847840547561646, -0.6895965337753296], [-1.216750979423523, -0.7031851410865784], [-1.1887990236282349, -0.6902034878730774], [-1.1341787576675415, -0.6628000736236572], [-1.0260283946990967, -0.5823454856872559], [-0.8671481013298035, -0.5011905431747437], [-0.6779059767723083, -0.38397160172462463], [-0.4705967307090759, -0.2751707434654236], [-0.23995713889598846, -0.14784327149391174], [-0.007516500540077686, -0.007170025259256363], [0.2270151525735855, 0.12194548547267914], [0.46399807929992676, 0.2683236300945282], [0.6689205169677734, 0.3869780898094177], [0.8519631028175354, 0.49726295471191406], [0.9937766194343567, 0.5840945839881897], [1.1168832778930664, 0.6605908870697021], [1.1799871921539307, 0.6928794980049133], [1.1997491121292114, 0.7073192596435547], [1.183134913444519, 0.707311749458313], [1.1233950853347778, 0.6570116281509399], [1.0231698751449585, 0.5875868797302246], [0.8657664656639099, 0.506422221660614], [0.6692233085632324, 0.3846440613269806], [0.473817378282547, 0.2788330316543579], [0.2514657974243164, 0.14104272425174713], [0.014038766734302044, 0.0058484203182160854], [-0.22657881677150726, -0.1335940957069397], [-0.44865086674690247, -0.27683502435684204], [-0.6573111414909363, -0.3801969587802887], [-0.8395320177078247, -0.49377599358558655], [-1.0012656450271606, -0.5854517221450806], [-1.0969823598861694, -0.6580348014831543], [-1.175423264503479, -0.7073549628257751], [-1.197190284729004, -0.7163649201393127], [-1.1757407188415527, -0.6940898895263672], [-1.1100050210952759, -0.6573671102523804], [-1.000625491142273, -0.5935719609260559], [-0.8471674919128418, -0.5015548467636108], [-0.6684359908103943, -0.3977413475513458], [-0.45837509632110596, -0.27832886576652527], [-0.2429955005645752, -0.1484076827764511], [-0.005536825396120548, -0.011206728406250477], [0.219736248254776, 0.13809193670749664], [0.4520837068557739, 0.2625894844532013], [0.6607790589332581, 0.3979478180408478], [0.838535726070404, 0.500688374042511], [0.9910354018211365, 0.6019392013549805], [1.0882501602172852, 0.643120527267456], [1.1651395559310913, 0.7077651619911194], [1.1925230026245117, 0.7078866958618164], [1.1780954599380493, 0.6994924545288086], [1.1001778841018677, 0.6667159795761108], [0.9880911111831665, 0.6008078455924988], [0.8489482402801514, 0.5053383111953735], [0.6671167612075806, 0.41221848130226135], [0.4466589093208313, 0.27159032225608826], [0.2366790771484375, 0.12520647048950195], [0.003591671120375395, 0.011858816258609295], [-0.2232104241847992, -0.13793900609016418], [-0.4404023289680481, -0.27240416407585144], [-0.6695858836174011, -0.3995613753795624], [-0.8482938408851624, -0.503136157989502], [-0.9740080833435059, -0.5915378928184509], [-1.0912203788757324, -0.6633108854293823], [-1.1560801267623901, -0.7164578437805176], [-1.1750402450561523, -0.7197344899177551], [-1.1633567810058594, -0.7022634744644165], [-1.079990029335022, -0.6672977209091187], [-0.9746465086936951, -0.5962929129600525], [-0.84001624584198, -0.5108168125152588], [-0.6557222008705139, -0.39885661005973816], [-0.4597158432006836, -0.27244460582733154], [-0.2357671558856964, -0.1476248949766159], [-0.013816107995808125, 0.0016020953189581633], [0.221178337931633, 0.13049136102199554], [0.4387274980545044, 0.27051135897636414], [0.6459128260612488, 0.40129467844963074], [0.8269401788711548, 0.5126367807388306], [0.9770050644874573, 0.5936358571052551], [1.0806865692138672, 0.6644716858863831], [1.1425628662109375, 0.7050032615661621], [1.1628587245941162, 0.7117317914962769], [1.141316533088684, 0.6927545070648193], [1.0796151161193848, 0.6616016626358032], [0.9735363721847534, 0.5961555242538452], [0.842597484588623, 0.5209397673606873], [0.6564100384712219, 0.39954861998558044], [0.44924020767211914, 0.278249591588974], [0.22897888720035553, 0.14335981011390686], [0.012965958565473557, 0.010616736486554146], [-0.2221204787492752, -0.1462865024805069], [-0.4392567574977875, -0.26835697889328003], [-0.640851616859436, -0.3906557261943817], [-0.8160728216171265, -0.5061532855033875], [-0.9683458209037781, -0.596099317073822], [-1.0730563402175903, -0.6598209738731384], [-1.1369011402130127, -0.6982887387275696], [-1.156190276145935, -0.7206663489341736], [-1.1324942111968994, -0.7041091322898865], [-1.0707800388336182, -0.6562251448631287], [-0.9668138027191162, -0.5986804962158203], [-0.8357369899749756, -0.515336275100708], [-0.6531003713607788, -0.4069322943687439], [-0.4378048777580261, -0.2868182957172394], [-0.24425892531871796, -0.1446675807237625], [-0.00610757153481245, 0.001675278414040804], [0.21370001137256622, 0.12886619567871094], [0.43321189284324646, 0.26246899366378784], [0.6354550123214722, 0.4068455100059509], [0.8067684173583984, 0.5164253115653992], [0.9410041570663452, 0.5931788682937622], [1.0665925741195679, 0.6624176502227783], [1.1181442737579346, 0.7249757647514343], [1.1377553939819336, 0.7187355160713196], [1.115635871887207, 0.7063297033309937], [1.0613795518875122, 0.6672228574752808], [0.9625604152679443, 0.6142269372940063], [0.8093653321266174, 0.5119706392288208], [0.6420954465866089, 0.41137486696243286], [0.4372180104255676, 0.27244246006011963], [0.22883082926273346, 0.15320080518722534], [0.0003128389944322407, 0.0034000228624790907], [-0.21873436868190765, -0.1369798630475998], [-0.4332524240016937, -0.26963257789611816], [-0.6330917477607727, -0.4034753143787384], [-0.8018967509269714, -0.5070346593856812], [-0.9407589435577393, -0.6031202673912048], [-1.0583029985427856, -0.6680802702903748], [-1.1165556907653809, -0.7090997099876404], [-1.1368292570114136, -0.7263314127922058], [-1.1002775430679321, -0.7075123190879822], [-1.0366926193237305, -0.6754088401794434], [-0.9391553997993469, -0.6067984104156494], [-0.8038890361785889, -0.5143957138061523], [-0.6253842115402222, -0.398494154214859], [-0.4390625059604645, -0.2726501226425171], [-0.2218092978000641, -0.1446031779050827], [-0.008457898162305355, -0.006767936050891876], [0.21813605725765228, 0.12892144918441772], [0.4256032109260559, 0.2694341838359833], [0.6162527799606323, 0.4057167172431946], [0.7920342087745667, 0.5036531686782837], [0.9250576496124268, 0.6021649241447449], [1.0288225412368774, 0.6609079241752625], [1.090388536453247, 0.7055323123931885], [1.1268913745880127, 0.7193558812141418], [1.0988432168960571, 0.7089419960975647], [1.0392566919326782, 0.6658454537391663], [0.9428435564041138, 0.5990117192268372], [0.7850019931793213, 0.5254042148590088], [0.6287816166877747, 0.40860694646835327], [0.4406457841396332, 0.26422494649887085], [0.2303326278924942, 0.13594910502433777], [0.00032238868880085647, 0.0035737911239266396], [-0.2177697718143463, -0.14388824999332428], [-0.4112188518047333, -0.27871280908584595], [-0.6179943680763245, -0.39827489852905273], [-0.7878497838973999, -0.5147069692611694], [-0.9108245372772217, -0.6050456166267395], [-1.026039481163025, -0.6632841229438782], [-1.0935367345809937, -0.7213941812515259], [-1.1056383848190308, -0.7104495763778687], [-1.0885450839996338, -0.7159764170646667], [-1.0247282981872559, -0.6667919158935547], [-0.9293761849403381, -0.5991973876953125], [-0.7898665070533752, -0.5103917717933655], [-0.6269710063934326, -0.4151492416858673], [-0.43212515115737915, -0.2824837863445282], [-0.2207091897726059, -0.14157435297966003], [-0.010322393849492073, 0.001516712480224669], [0.21272221207618713, 0.1437193602323532], [0.41588208079338074, 0.2770111560821533], [0.6006070375442505, 0.3961944878101349], [0.7754278182983398, 0.5204446315765381], [0.907534122467041, 0.5971079468727112], [1.013768196105957, 0.6677848100662231], [1.0696853399276733, 0.7182282209396362], [1.087576985359192, 0.7258459329605103], [1.084664225578308, 0.717967689037323], [1.01841402053833, 0.6596673130989075], [0.9139615893363953, 0.5996080636978149], [0.7766156792640686, 0.4998810291290283], [0.6215436458587646, 0.407098650932312], [0.4305218458175659, 0.2751874029636383], [0.21739788353443146, 0.14924506843090057], [0.008554978296160698, -0.0005467506125569344], [-0.20343329012393951, -0.14817959070205688], [-0.41108059883117676, -0.2682185173034668], [-0.6004812121391296, -0.4122453033924103], [-0.7636377215385437, -0.5121445059776306], [-0.9053016901016235, -0.6007903218269348], [-1.0080043077468872, -0.6765069961547852], [-1.055858850479126, -0.7079922556877136], [-1.0921497344970703, -0.7233923673629761], [-1.0640186071395874, -0.7082511186599731], [-0.9991552233695984, -0.6768922209739685], [-0.9055365920066833, -0.6119200587272644], [-0.7710386514663696, -0.5185445547103882], [-0.6100068688392639, -0.4056835174560547], [-0.43873345851898193, -0.28305113315582275], [-0.21354719996452332, -0.14194512367248535], [-0.0012497941497713327, -0.00604379503056407], [0.20919345319271088, 0.14099335670471191], [0.41773921251296997, 0.2640755772590637], [0.5985796451568604, 0.4007377326488495], [0.7511067390441895, 0.5140665173530579], [0.8971245884895325, 0.6055951118469238], [0.9909711480140686, 0.677279531955719], [1.058140754699707, 0.722235918045044], [1.0689197778701782, 0.7301939129829407], [1.058315634727478, 0.7162899374961853], [0.9939995408058167, 0.6845260858535767], [0.8974868059158325, 0.6058484315872192], [0.7619758248329163, 0.5275879502296448], [0.6122860908508301, 0.41125908493995667], [0.41294988989830017, 0.2948566973209381], [0.20551227033138275, 0.14881360530853271], [0.003971941769123077, 0.0015882609877735376], [-0.1920136958360672, -0.1365227848291397], [-0.40797358751296997, -0.26824304461479187], [-0.5818911790847778, -0.39663708209991455], [-0.7477642893791199, -0.5168595314025879], [-0.8963745832443237, -0.6173840165138245], [-0.9921122193336487, -0.6732699275016785], [-1.0504850149154663, -0.7229159474372864], [-1.0794168710708618, -0.7335533499717712], [-1.0576444864273071, -0.7224157452583313], [-0.9987121224403381, -0.6841720938682556], [-0.8905482888221741, -0.6051323413848877], [-0.7587014436721802, -0.5227888226509094], [-0.6045103073120117, -0.41135579347610474], [-0.41464218497276306, -0.29107579588890076], [-0.20977608859539032, -0.15436691045761108], [-0.003103086492046714, -0.006001488771289587], [0.19702379405498505, 0.13900676369667053], [0.40938636660575867, 0.28652143478393555], [0.5821533203125, 0.4064621329307556], [0.7446442246437073, 0.511353611946106], [0.8913004398345947, 0.6124934554100037], [0.9808244705200195, 0.6824339628219604], [1.0553269386291504, 0.721871018409729], [1.0689618587493896, 0.7416834235191345], [1.046976923942566, 0.7252234220504761], [0.9772410988807678, 0.6867315173149109], [0.8773366212844849, 0.6032322645187378], [0.7589537501335144, 0.5208890438079834], [0.5879006385803223, 0.41038981080055237], [0.4130942225456238, 0.2887858748435974], [0.2136642187833786, 0.15111380815505981], [0.009823421947658062, -0.000544628594070673], [-0.2021753340959549, -0.1332997828722], [-0.3988068699836731, -0.2830688953399658], [-0.5867122411727905, -0.4049846827983856], [-0.7425273060798645, -0.5182624459266663], [-0.8763160705566406, -0.6119139194488525], [-0.971682071685791, -0.6821339130401611], [-1.036911964416504, -0.7152040004730225], [-1.060840129852295, -0.7457779049873352], [-1.0425297021865845, -0.7283823490142822], [-0.9755871295928955, -0.6879556775093079], [-0.8856620788574219, -0.6239264011383057], [-0.7528162598609924, -0.524287223815918], [-0.5910255908966064, -0.41813719272613525], [-0.4076833426952362, -0.2895086407661438], [-0.20948651432991028, -0.15486419200897217], [-0.003944461699575186, 0.0023168178740888834], [0.21690736711025238, 0.13554376363754272], [0.4008747637271881, 0.2757405638694763], [0.5837252736091614, 0.41179579496383667], [0.7349532842636108, 0.5202081799507141], [0.8793410062789917, 0.6146470904350281], [0.9827525615692139, 0.6817362904548645], [1.0308303833007812, 0.7254902720451355], [1.0541085004806519, 0.7508072853088379], [1.0446248054504395, 0.7422896027565002], [0.9823412299156189, 0.6961185336112976], [0.8886748552322388, 0.62713223695755], [0.7487108111381531, 0.5391474962234497], [0.598963737487793, 0.42628011107444763], [0.41432687640190125, 0.28891444206237793], [0.21822252869606018, 0.1568804830312729], [0.005874942056834698, -0.009126872755587101], [-0.20022757351398468, -0.13646678626537323], [-0.39538443088531494, -0.27842289209365845], [-0.584288477897644, -0.4088311493396759], [-0.7283703684806824, -0.529312789440155], [-0.8585792779922485, -0.6245002150535583], [-0.959252655506134, -0.6898913979530334], [-1.0235573053359985, -0.7412188649177551], [-1.042127251625061, -0.7502961754798889], [-1.0256776809692383, -0.7360512614250183], [-0.9784969687461853, -0.6977652311325073], [-0.875266432762146, -0.6297025680541992], [-0.7433066964149475, -0.5417726635932922], [-0.5900404453277588, -0.41888588666915894], [-0.4081478714942932, -0.2940751910209656], [-0.2240493893623352, -0.1461304873228073], [-0.004276718478649855, -0.002073141746222973], [0.21298077702522278, 0.14293918013572693], [0.38831788301467896, 0.2870592772960663], [0.570306122303009, 0.4196264445781708], [0.7340212464332581, 0.530278205871582], [0.8604092597961426, 0.6248131394386292], [0.961636483669281, 0.6969848871231079], [1.0262380838394165, 0.7473889589309692], [1.047400712966919, 0.7567986845970154], [1.0245225429534912, 0.7513595223426819], [0.970607340335846, 0.7153781056404114], [0.8696725964546204, 0.6384782195091248], [0.7455124855041504, 0.5449253916740417], [0.5829222798347473, 0.435687392950058], [0.41125282645225525, 0.29500097036361694], [0.21542510390281677, 0.15382541716098785], [0.01287416834384203, -0.00209578825160861], [-0.1994461864233017, -0.14391344785690308], [-0.3895907700061798, -0.2977578043937683], [-0.5798414945602417, -0.42018505930900574], [-0.7227157354354858, -0.5418704152107239], [-0.8755385279655457, -0.6333103179931641], [-0.9663843512535095, -0.6934499740600586], [-1.0206186771392822, -0.7479550838470459], [-1.0231252908706665, -0.7596181631088257], [-1.015297532081604, -0.7450780868530273], [-0.9697558879852295, -0.7046529650688171], [-0.8757228851318359, -0.6377172470092773], [-0.7322357296943665, -0.5504246950149536], [-0.5837631821632385, -0.42329737544059753], [-0.4030943512916565, -0.3127935230731964], [-0.20698070526123047, -0.1567559540271759], [-0.0017949637258425355, -0.013171316124498844], [0.20186753571033478, 0.14369089901447296], [0.40392962098121643, 0.2943834960460663], [0.5732974410057068, 0.4165303707122803], [0.7251493334770203, 0.5430402755737305], [0.8581597208976746, 0.6334891319274902], [0.955646812915802, 0.7096385955810547], [1.019925832748413, 0.7401401400566101], [1.0379464626312256, 0.7574563026428223], [1.0177538394927979, 0.7555786371231079], [0.9588885307312012, 0.7125658392906189], [0.870176374912262, 0.6389561295509338], [0.7216811776161194, 0.5422095060348511], [0.5911328792572021, 0.44054460525512695], [0.39527732133865356, 0.2930518090724945], [0.20879501104354858, 0.1595558375120163], [0.015341812744736671, -0.002172744832932949], [-0.18092471361160278, -0.14382387697696686], [-0.3930453360080719, -0.2917534112930298], [-0.5668714642524719, -0.43553805351257324], [-0.7250082492828369, -0.5312616229057312], [-0.8620740175247192, -0.6382338404655457], [-0.9575085043907166, -0.7119787931442261], [-1.0067006349563599, -0.7598115801811218], [-1.0410293340682983, -0.7688201665878296], [-1.013986587524414, -0.7635498046875], [-0.9508301615715027, -0.7255256772041321], [-0.8498436212539673, -0.6444699764251709], [-0.7309013605117798, -0.5557597279548645], [-0.5753337740898132, -0.4301157593727112], [-0.40198853611946106, -0.30716243386268616], [-0.2123376876115799, -0.15444019436836243], [-0.00019065034575760365, -0.001727377763018012], [0.1993236541748047, 0.1619834452867508], [0.3956010937690735, 0.29953619837760925], [0.5620929598808289, 0.42629939317703247], [0.7272958159446716, 0.547382652759552], [0.846795916557312, 0.6475515961647034], [0.9418112635612488, 0.7145577073097229], [1.0115278959274292, 0.769531786441803], [1.0245717763900757, 0.7821966409683228], [1.022019863128662, 0.7697252035140991], [0.9429584741592407, 0.7274641394615173], [0.8502684235572815, 0.6494495272636414], [0.7373771667480469, 0.5416364669799805], [0.5712442994117737, 0.4311586618423462], [0.3932747542858124, 0.3086545169353485], [0.19415542483329773, 0.16678136587142944], [-0.0018451788928359747, -0.0028114886954426765], [-0.19994492828845978, -0.15842586755752563], [-0.39030084013938904, -0.299504816532135], [-0.5831283926963806, -0.42958691716194153], [-0.7199821472167969, -0.5549931526184082], [-0.848341703414917, -0.6422375440597534], [-0.9428241848945618, -0.7148706316947937], [-1.0027709007263184, -0.7779523134231567], [-1.0212727785110474, -0.7893257141113281], [-1.0061283111572266, -0.773725152015686], [-0.9481223225593567, -0.7365700006484985], [-0.8572415113449097, -0.6661425232887268], [-0.7139865756034851, -0.5578746199607849], [-0.5677539706230164, -0.4318111836910248], [-0.40690964460372925, -0.3066190779209137], [-0.20249737799167633, -0.16022464632987976], [-0.007495271973311901, -0.012784700840711594], [0.1859935224056244, 0.14733998477458954], [0.38853684067726135, 0.3047219216823578], [0.5646427869796753, 0.4344228506088257], [0.7249528765678406, 0.553729772567749], [0.8500991463661194, 0.6556606292724609], [0.927613377571106, 0.7284117937088013], [0.9991174936294556, 0.7780787348747253], [1.0067511796951294, 0.7978468537330627], [1.005149006843567, 0.7887449264526367], [0.9382824301719666, 0.733310341835022], [0.8562381267547607, 0.6587620973587036], [0.7295780777931213, 0.5771100521087646], [0.5808345079421997, 0.43956229090690613], [0.39845186471939087, 0.3081749379634857], [0.1933366060256958, 0.16755332052707672], [0.009712204337120056, 0.0070573813281953335], [-0.18976512551307678, -0.149171382188797], [-0.3688123822212219, -0.29800114035606384], [-0.5644116997718811, -0.4330572187900543], [-0.7082115411758423, -0.550402820110321], [-0.837774395942688, -0.6404779553413391], [-0.9305328130722046, -0.7421540021896362], [-0.9987578392028809, -0.7769582867622375], [-1.016324758529663, -0.7955623865127563], [-1.0030391216278076, -0.7855895757675171], [-0.9458119869232178, -0.7406035661697388], [-0.8498315811157227, -0.660688579082489], [-0.7247061729431152, -0.5712851285934448], [-0.5621048808097839, -0.449472576379776], [-0.40168869495391846, -0.3086231052875519], [-0.2008271962404251, -0.16462595760822296], [-0.00489798653870821, 0.0023716206196695566], [0.18878117203712463, 0.15514372289180756], [0.37952348589897156, 0.29950740933418274], [0.5586909055709839, 0.4418815076351166], [0.7148863077163696, 0.5719953179359436], [0.8474750518798828, 0.6551449298858643], [0.9348737001419067, 0.7443328499794006], [0.9903982877731323, 0.7929822206497192], [1.0221774578094482, 0.7999051213264465], [0.9942266941070557, 0.7901486158370972], [0.9465035796165466, 0.7452979683876038], [0.851387619972229, 0.6642128229141235], [0.718612790107727, 0.5825142860412598], [0.5793322324752808, 0.4468885660171509], [0.39367014169692993, 0.32960245013237], [0.2007020115852356, 0.15859057009220123], [0.018026696518063545, 0.001964439870789647], [-0.1913243979215622, -0.1684902161359787], [-0.3713582158088684, -0.3080853521823883], [-0.5719882249832153, -0.43000033497810364], [-0.7136141061782837, -0.568800151348114], [-0.8447557687759399, -0.6782087683677673], [-0.9351025819778442, -0.7408123016357422], [-0.9807904362678528, -0.7977882623672485], [-1.0148288011550903, -0.8190782070159912], [-0.9941213726997375, -0.7910149693489075], [-0.9446859955787659, -0.7501909732818604], [-0.839497447013855, -0.6862747669219971], [-0.7142521142959595, -0.583639919757843], [-0.5642268657684326, -0.4548064172267914], [-0.38692256808280945, -0.3082022964954376], [-0.19809222221374512, -0.1602170467376709], [-0.01044880598783493, -0.00535223912447691], [0.20134000480175018, 0.1621861606836319], [0.372105211019516, 0.31056585907936096], [0.5580358505249023, 0.46316879987716675], [0.7031474113464355, 0.5710671544075012], [0.8304893970489502, 0.6866938471794128], [0.9301836490631104, 0.7520532608032227], [0.9890827536582947, 0.8041386604309082], [1.0153192281723022, 0.8066833019256592], [0.9947418570518494, 0.8083328604698181], [0.914257287979126, 0.7427603602409363], [0.8401272892951965, 0.689110279083252], [0.7177062034606934, 0.575221836566925], [0.5624493360519409, 0.4467012882232666], [0.38835233449935913, 0.31183162331581116], [0.19123458862304688, 0.17258790135383606], [0.006693118251860142, -0.00043966781231574714], [-0.1859855055809021, -0.16304706037044525], [-0.37172040343284607, -0.3143615424633026], [-0.5588890314102173, -0.4469803273677826], [-0.7063964009284973, -0.5762534737586975], [-0.8288654088973999, -0.6792886853218079], [-0.9162660241127014, -0.756130039691925], [-0.9876516461372375, -0.8123371601104736], [-1.0121383666992188, -0.8150203824043274], [-0.9835427403450012, -0.808134138584137], [-0.935110867023468, -0.7585044503211975], [-0.8329085111618042, -0.6839672923088074], [-0.7074963450431824, -0.5875964164733887], [-0.5613369345664978, -0.45337653160095215], [-0.3977546691894531, -0.3166916072368622], [-0.19193917512893677, -0.15850667655467987], [-0.009157364256680012, -0.00929259043186903], [0.1885111927986145, 0.1565980464220047], [0.3795108199119568, 0.3138033449649811], [0.5527335405349731, 0.454984575510025], [0.7033792734146118, 0.5804120898246765], [0.8361661434173584, 0.680304765701294], [0.9274157881736755, 0.7581294178962708], [0.9748891592025757, 0.7994097471237183], [0.9937692284584045, 0.8231543898582458], [0.9795423746109009, 0.8204162120819092], [0.9305547475814819, 0.7658862471580505], [0.8361911773681641, 0.6995289325714111], [0.713416337966919, 0.5872126817703247], [0.5620241761207581, 0.4613719582557678], [0.3831046223640442, 0.3148112893104553], [0.2104852944612503, 0.16782663762569427], [0.009658762253820896, -0.007461169734597206], [-0.18199753761291504, -0.16111832857131958], [-0.3748229444026947, -0.30465167760849], [-0.5443712472915649, -0.4607667922973633], [-0.7007912397384644, -0.5853292942047119], [-0.8216904997825623, -0.6920881867408752], [-0.9064595103263855, -0.7584425210952759], [-0.9814666509628296, -0.8131632804870605], [-0.9943072199821472, -0.8313947916030884], [-0.9773156046867371, -0.8161607980728149], [-0.9362655878067017, -0.7773913145065308], [-0.8334019184112549, -0.6972261667251587], [-0.7087676525115967, -0.5937709808349609], [-0.5604408979415894, -0.4782707095146179], [-0.3890254497528076, -0.31579267978668213], [-0.1992608904838562, -0.16642391681671143], [-0.004550592973828316, 0.00883396901190281], [0.18334554135799408, 0.16639316082000732], [0.37411877512931824, 0.312953919172287], [0.5594526529312134, 0.4617561101913452], [0.7032052874565125, 0.5882119536399841], [0.8225303292274475, 0.6888851523399353], [0.9303579330444336, 0.7754371762275696], [0.971169650554657, 0.8272860050201416], [0.9983440637588501, 0.8443887829780579], [0.9778339862823486, 0.8282264471054077], [0.9125216603279114, 0.7742407321929932], [0.8262330293655396, 0.7062069773674011], [0.7140706181526184, 0.5972817540168762], [0.5452902317047119, 0.47287604212760925], [0.3890952169895172, 0.3284684121608734], [0.1992536038160324, 0.17316804826259613], [0.011721535585820675, -0.00476358737796545], [-0.18319112062454224, -0.16744227707386017], [-0.3688313663005829, -0.31197670102119446], [-0.5469650030136108, -0.4674113988876343], [-0.6942439675331116, -0.593708336353302], [-0.8226345777511597, -0.7070220112800598], [-0.9139137864112854, -0.7967911958694458], [-0.975341796875, -0.8291758298873901], [-1.0009063482284546, -0.8511704802513123], [-0.9792059063911438, -0.8442497253417969], [-0.9214580655097961, -0.7991446852684021], [-0.8371684551239014, -0.6976119875907898], [-0.7016669511795044, -0.6134456396102905], [-0.5502169132232666, -0.482485830783844], [-0.3896901309490204, -0.33038994669914246], [-0.19959139823913574, -0.16706956923007965], [-0.003312661312520504, -0.003895960282534361], [0.18732097744941711, 0.14721457660198212], [0.3785313069820404, 0.33090096712112427], [0.5492956638336182, 0.46471571922302246], [0.6985465288162231, 0.5996372103691101], [0.8303543925285339, 0.7120679616928101], [0.9024578332901001, 0.7892478108406067], [0.9775997400283813, 0.8490999341011047], [0.9886318445205688, 0.8686556816101074], [0.9794668555259705, 0.842614471912384], [0.9228882789611816, 0.8097236752510071], [0.8184456825256348, 0.7167249917984009], [0.7102948427200317, 0.6107960939407349], [0.5485497117042542, 0.48013779520988464], [0.37184014916419983, 0.33477529883384705], [0.19677124917507172, 0.16932591795921326], [0.0005439292872324586, -0.004011157434433699], [-0.19897155463695526, -0.15596367418766022], [-0.3695322871208191, -0.32702741026878357], [-0.5550075769424438, -0.46990928053855896], [-0.7006989121437073, -0.6091444492340088], [-0.8234454989433289, -0.7126991748809814], [-0.9119628667831421, -0.7905683517456055], [-0.974786639213562, -0.8597496151924133], [-0.9905794262886047, -0.8737292289733887], [-0.9834994077682495, -0.8536468744277954], [-0.9198136329650879, -0.8022863268852234], [-0.8162718415260315, -0.731732189655304], [-0.7129129767417908, -0.6136952638626099]]]}' http://10.0.1.8:5001/invocations
print(type(answer))

index = answer.l[len(answer.l)-1]
index = index.replace('[','')
index = index.replace(']','')
print(f'Predicted class is {classes[eval(index).index(max(eval(index)))]}')

In [ ]:
#Replace IP of host with adress of your MLflow host machine
#Replace port with serving model port 
host = '89.208.197.151'
port = '5001'

url = f'http://{host}:{port}/invocations'

headers = {
    'Content-Type': 'application/json',
}

# test_x is a Pandas dataframe with data for testing the ML model
# http_data = input_example.to_json()

r = requests.post(url=url, headers=headers, data=input_example)
response = json.loads(r.content)

print(f'Predictions: {r.text}')
print(f'Predicted class is {classes[response[0].index(max(response[0]))]}')